In [ ]:
!pip install cgmquantify

In [ ]:
!pip install imbalanced-learn


In [ ]:
import cgmquantify as cgm
import pandas as pd
import datetime as datetime
from datetime import date
from scipy.stats import stats
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from scipy.signal import find_peaks
from scipy.stats import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd


## Read dataframe with labeled by patient_reported_outcome

In [ ]:
df_merged_all = pd.read_csv('df_overnight_and_PRO.csv', sep='\t')

In [ ]:
df_merged_all.label.value_counts()

In [ ]:
df_merged_all = df_merged_all.dropna(subset=['axis1', 'axis2', 'axis3'],how='all')

df_merged_all['axis1'] = df_merged_all['axis1'].fillna(0)
df_merged_all['axis2'] = df_merged_all['axis2'].fillna(0)
df_merged_all['axis3'] = df_merged_all['axis3'].fillna(0)

df_merged_all['time'] = pd.to_datetime(df_merged_all['time'] ,errors = 'coerce')

df_merged_all = df_merged_all.reset_index(drop=True)
df_merged_all = df_merged_all.drop(["Unnamed: 0", "joinID"], axis=1)

fasting_states_to_keep = ['fasting', 'non-fasting']
df_merged_all = df_merged_all[df_merged_all.label.isin(fasting_states_to_keep)]


final_df = df_merged_all.sort_values(by=['id', 'time'], ascending=True)

In [ ]:
final_df['Time'] = pd.to_datetime(final_df['time'], format='%Y-%m-%dT%H:%M:%S')
final_df['Glucose'] = pd.to_numeric(final_df['gl'])
final_df['Day'] = final_df["Time"].dt.date
final_df = final_df.reset_index(drop=True)

## Split to windowed data

In [ ]:
x_list_train = []
train_labels = []


window_size = 3
step_size = 1

# creating overlaping windows of size window-size 100
for i in range(0, df_train.shape[0] - window_size, step_size):
    time = df_train['Time'].values[i: i + window_size]
    times = df_train['Day'].values[i: i + window_size]
    gls = df_train['Glucose'].values[i: i + window_size]
    steps = df_train['steps'].values[i: i + window_size]
    inclineStanding = df_train['inclineStanding'].values[i: i + window_size]
    inclineSitting = df_train['inclineSitting'].values[i: i + window_size]
    inclineLying = df_train['inclineLying'].values[i: i + window_size]
    
    timedelta_first = time[1] - time[0]
    timedelta_second = time[2] - time[1]
    one_hour = 3600000000000
    
    if (timedelta_first < one_hour and timedelta_second < one_hour):
   
    
        xs = df_train['axis1'].values[i: i + window_size]
        ys = df_train['axis2'].values[i: i + window_size]
        zs = df_train['axis3'].values[i: i + window_size]

        label = stats.mode(df_train['label'][i: i + window_size])[0][0]

        df_slice = pd.DataFrame(time, columns=['Time'])
        df_slice["Day"] = times
        df_slice["Glucose"] = gls   
        df_slice["axis1"] = xs
        df_slice["axis2"] = ys
        df_slice["axis3"] = zs
        df_slice["steps"] = steps
        df_slice["inclineStanding"] = inclineStanding
        df_slice["inclineSitting"] = inclineSitting
        df_slice["inclineLying"] = inclineLying

        x_list_train.append(df_slice)

        train_labels.append(label)

In [ ]:
x_list_test = []
test_labels = []


window_size = 3
step_size = 1

# creating overlaping windows of size window-size 100
for i in range(0, df_test.shape[0] - window_size, step_size):
    time = df_test['Time'].values[i: i + window_size]
    times = df_test['Day'].values[i: i + window_size]
    gls = df_test['Glucose'].values[i: i + window_size]
    steps = df_test['steps'].values[i: i + window_size]
    inclineStanding = df_test['inclineStanding'].values[i: i + window_size]
    inclineSitting = df_test['inclineSitting'].values[i: i + window_size]
    inclineLying = df_test['inclineLying'].values[i: i + window_size]
    
    timedelta_first = time[1] - time[0]
    timedelta_second = time[2] - time[1]
    one_hour = 3600000000000
    
    if (timedelta_first < one_hour and timedelta_second < one_hour):
    
        xs = df_test['axis1'].values[i: i + window_size]
        ys = df_test['axis2'].values[i: i + window_size]
        zs = df_test['axis3'].values[i: i + window_size]

        label = stats.mode(df_test['label'][i: i + window_size])[0][0]

        df_slice = pd.DataFrame(time, columns=['Time'])
        df_slice["Day"] = times
        df_slice["Glucose"] = gls     
        df_slice["axis1"] = xs
        df_slice["axis2"] = ys
        df_slice["axis3"] = zs
        df_slice["steps"] = steps
        df_slice["inclineStanding"] = inclineStanding
        df_slice["inclineSitting"] = inclineSitting
        df_slice["inclineLying"] = inclineLying

        x_list_test.append(df_slice)

        test_labels.append(label)

## Feature computation

In [ ]:
def feat_statistical_measures_gl_acc(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        cgm_summary = list(cgm.summary(df_temp))
        cgm_LBGI = cgm.LBGI(df_temp)
        cgm_HBGI = cgm.HBGI(df_temp)
        cgm_ADRR = cgm.ADRR(df_temp)
        cgm_GMI = cgm.GMI(df_temp)
        cgm_J_index = cgm.J_index(df_temp)
        cgm_eA1c = cgm.eA1c(df_temp)
        cgm_interdaysd = cgm.interdaysd(df_temp)
        cgm_interdaycv = cgm.interdaycv(df_temp)
        cgm_TOR = cgm.TOR(df_temp, sd=1, sr=15)
        cgm_TIR = cgm.TIR(df_temp, sd=1, sr=15)
        cgm_POR = cgm.POR(df_temp, sd=1, sr=15)
        cgm_MAGN = cgm.MAGN(df_temp, sd=1)
        cgm_MAGE = cgm.MAGE(df_temp, sd=1)  
        
        cgm_summary.append(cgm_LBGI)
        cgm_summary.append(cgm_HBGI)
        cgm_summary.append(cgm_ADRR)
        cgm_summary.append(cgm_GMI)
        cgm_summary.append(cgm_J_index)
        cgm_summary.append(cgm_eA1c)
        cgm_summary.append(cgm_interdaysd)
        cgm_summary.append(cgm_interdaycv)

        cgm_summary.append(cgm_TOR)
        cgm_summary.append(cgm_TIR)
        cgm_summary.append(cgm_POR)
        
        
        # mean
        x_mean = df_temp["axis1"].mean()
        y_mean = df_temp["axis2"].mean()
        z_mean = df_temp["axis3"].mean()

        # std dev
        x_std = df_temp["axis1"].std()
        y_std = df_temp["axis2"].std()
        z_std = df_temp["axis3"].std()
        
        
        # avg absolute diff
        x_aad = (df_temp["axis1"] - df_temp["axis1"].mean()).abs().mean()
        y_aad = (df_temp["axis2"] - df_temp["axis2"].mean()).abs().mean()
        z_aad = (df_temp["axis3"] - df_temp["axis3"].mean()).abs().mean()

                                                 
        # min
        x_min = df_temp["axis1"].min()
        y_min = df_temp["axis2"].min()
        z_min = df_temp["axis3"].min()

        # max
        x_max = df_temp["axis1"].max()
        y_max = df_temp["axis2"].max()
        z_max = df_temp["axis3"].max()

        # max-min diff
        x_maxmin_diff = x_max - x_min
        y_maxmin_diff = y_max - y_min
        z_maxmin_diff = z_max - z_min

        # median
        x_median = df_temp["axis1"].median()
        y_median = df_temp["axis2"].median()
        z_median = df_temp["axis3"].median()
        steps = df_temp["steps"].median()
        inclineStanding = df_temp["inclineStanding"].median()
        inclineSitting = df_temp["inclineSitting"].median()
        inclineLying = df_temp["inclineLying"].median()
        
        # median abs dev
        x_mad = (df_temp["axis1"] - df_temp["axis1"].median()).abs().median()
        y_mad = (df_temp["axis2"] - df_temp["axis2"].median()).abs().median()
        z_mad = (df_temp["axis3"] - df_temp["axis3"].median()).abs().median()

        # interquartile range
        x_IQR = (df_temp["axis1"].quantile(0.75) - df_temp["axis1"].quantile(0.25))
        y_IQR = (df_temp["axis2"].quantile(0.75) - df_temp["axis2"].quantile(0.25))
        z_IQR = (df_temp["axis3"].quantile(0.75) - df_temp["axis3"].quantile(0.25))

        # negtive count
        x_neg_count = (df_temp["axis1"] < 0).sum().sum()
        y_neg_count = (df_temp["axis2"] < 0).sum().sum()
        z_neg_count = (df_temp["axis3"] < 0).sum().sum()

        # positive count
        x_pos_count = (df_temp["axis1"] > 0).sum().sum()
        y_pos_count = (df_temp["axis2"] > 0).sum().sum()
        z_pos_count = (df_temp["axis3"] > 0).sum().sum()

        # values above mean
        x_above_mean = (df_temp["axis1"] > df_temp["axis1"].mean()).sum().sum()
        y_above_mean = (df_temp["axis2"] > df_temp["axis2"].mean()).sum().sum()
        z_above_mean = (df_temp["axis3"] > df_temp["axis3"].mean()).sum().sum()

        # number of peaks
        x_peak_count = len(find_peaks(df_temp["axis1"])[0])
        y_peak_count = len(find_peaks(df_temp["axis2"])[0])
        z_peak_count = len(find_peaks(df_temp["axis3"])[0])

        # skewness
        x_skewness = stats.skew(df_temp["axis1"])
        y_skewness = stats.skew(df_temp["axis2"])
        z_skewness = stats.skew(df_temp["axis3"])

        # kurtosis
        x_kurtosis = stats.kurtosis(df_temp["axis1"])
        y_kurtosis = stats.kurtosis(df_temp["axis2"])
        z_kurtosis = stats.kurtosis(df_temp["axis3"])

        # energy
        x_energy = ((df_temp["axis1"] **2) / 100).sum()
        y_energy = ((df_temp["axis2"] **2) / 100).sum()
        z_energy = ((df_temp["axis3"] **2) / 100).sum()

        # avg resultant
        avg_result_accl = ((df_temp["axis1"] ** 2 + df_temp["axis2"] ** 2 + df_temp["axis3"] ** 2) ** 0.5).mean()

        # signal magnitude area
        sma = (df_temp["axis1"].abs() / 100).sum() +(df_temp["axis2"].abs() / 100).sum() + (df_temp["axis3"].abs() / 100).sum()
        
        


        acc_summary = []
        acc_summary.extend([x_mean , y_mean, z_mean, x_std, y_std, z_std, x_aad, y_aad, z_aad, x_min, y_min, z_min, x_max, y_max, z_max, 
                            x_maxmin_diff, y_maxmin_diff, z_maxmin_diff, x_median, y_median, z_median, x_mad, y_mad, z_mad, x_IQR, y_IQR, z_IQR,
                            x_neg_count, y_neg_count, z_neg_count, x_pos_count, y_pos_count, z_pos_count,x_above_mean, y_above_mean, z_above_mean,
                            x_peak_count, y_peak_count, z_peak_count, x_skewness, y_skewness, z_skewness, x_kurtosis, y_kurtosis, z_kurtosis, x_energy,
                            y_energy, z_energy, avg_result_accl, sma, steps, inclineStanding, inclineSitting, inclineLying])
        

        features = cgm_summary
        features.extend(acc_summary)

        X_train_temp = pd.DataFrame([features], columns = ["mean", "median","minimum", "maximum","first_quartile", "third_quartile",
                                                              "LBGI", "HBGI", "ADRR","GMI", "J_index", "eA1c", "interdaysd",
                                                              "cgm_interdaycv", 
                                                              "cgm_TOR", "cgm_TIR",
                                                              "cgm_POR",
                                                              "x_mean" , "y_mean", "z_mean", "x_std", "y_std", "z_std", "x_aad", "y_aad", "z_aad", "x_min", "y_min", "z_min", "x_max", "y_max", "z_max", 
                                                              "x_maxmin_diff", "y_maxmin_diff", "z_maxmin_diff", "x_median", "y_median", "z_median", "x_mad", "y_mad", "z_mad", "x_IQR", "y_IQR", "z_IQR",
                                                              "x_neg_count", "y_neg_count", "z_neg_count", "x_pos_count", "y_pos_count", "z_pos_count","x_above_mean", "y_above_mean", "z_above_mean",
                                                              "x_peak_count", "y_peak_count", "z_peak_count", "x_skewness", "y_skewness", "z_skewness", "x_kurtosis", "y_kurtosis", "z_kurtosis", "x_energy",
                                                              "y_energy", "z_energy", "avg_result_accl", "sma", "steps", "inclineStanding", "inclineSitting", "inclineLying"])

        
        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)
        
    return X_train

In [ ]:
def feat_statistical_measures_gl(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        cgm_summary = list(cgm.summary(df_temp))
        cgm_LBGI = cgm.LBGI(df_temp)
        cgm_HBGI = cgm.HBGI(df_temp)
        cgm_ADRR = cgm.ADRR(df_temp)
        cgm_GMI = cgm.GMI(df_temp)
        cgm_J_index = cgm.J_index(df_temp)
        cgm_eA1c = cgm.eA1c(df_temp)
        cgm_interdaysd = cgm.interdaysd(df_temp)
        cgm_interdaycv = cgm.interdaycv(df_temp)
        cgm_TOR = cgm.TOR(df_temp, sd=1, sr=15)
        cgm_TIR = cgm.TIR(df_temp, sd=1, sr=15)
        cgm_POR = cgm.POR(df_temp, sd=1, sr=15)

        
        cgm_summary.append(cgm_LBGI)
        cgm_summary.append(cgm_HBGI)
        cgm_summary.append(cgm_ADRR)
        cgm_summary.append(cgm_GMI)
        cgm_summary.append(cgm_J_index)
        cgm_summary.append(cgm_eA1c)
        cgm_summary.append(cgm_interdaysd)
        cgm_summary.append(cgm_interdaycv)
        cgm_summary.append(cgm_TOR)
        cgm_summary.append(cgm_TIR)
        cgm_summary.append(cgm_POR)


        features = cgm_summary

        X_train_temp = pd.DataFrame([cgm_summary], columns = ["mean", "median","minimum", "maximum","first_quartile", "third_quartile",
                                                              "LBGI", "HBGI", "ADRR","GMI", "J_index", "eA1c", "interdaysd",
                                                              "cgm_interdaycv", 
                                                              "cgm_TOR", "cgm_TIR",
                                                              "cgm_POR"])

        
        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)
        
    return X_train

In [ ]:
def feat_statistical_measures_acc(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:

        # mean
        x_mean = df_temp["axis1"].mean()
        y_mean = df_temp["axis2"].mean()
        z_mean = df_temp["axis3"].mean()

        # std dev
        x_std = df_temp["axis1"].std()
        y_std = df_temp["axis2"].std()
        z_std = df_temp["axis3"].std()
        
        
        # avg absolute diff
        x_aad = (df_temp["axis1"] - df_temp["axis1"].mean()).abs().mean()
        y_aad = (df_temp["axis2"] - df_temp["axis2"].mean()).abs().mean()
        z_aad = (df_temp["axis3"] - df_temp["axis3"].mean()).abs().mean()

                                                 
        # min
        x_min = df_temp["axis1"].min()
        y_min = df_temp["axis2"].min()
        z_min = df_temp["axis3"].min()

        # max
        x_max = df_temp["axis1"].max()
        y_max = df_temp["axis2"].max()
        z_max = df_temp["axis3"].max()

        # max-min diff
        x_maxmin_diff = x_max - x_min
        y_maxmin_diff = y_max - y_min
        z_maxmin_diff = z_max - z_min

        # median
        x_median = df_temp["axis1"].median()
        y_median = df_temp["axis2"].median()
        z_median = df_temp["axis3"].median()
        steps = df_temp["steps"].median()
        inclineStanding = df_temp["inclineStanding"].median()
        inclineSitting = df_temp["inclineSitting"].median()
        inclineLying = df_temp["inclineLying"].median()
        
        # median abs dev
        x_mad = (df_temp["axis1"] - df_temp["axis1"].median()).abs().median()
        y_mad = (df_temp["axis2"] - df_temp["axis2"].median()).abs().median()
        z_mad = (df_temp["axis3"] - df_temp["axis3"].median()).abs().median()

        # interquartile range
        x_IQR = (df_temp["axis1"].quantile(0.75) - df_temp["axis1"].quantile(0.25))
        y_IQR = (df_temp["axis2"].quantile(0.75) - df_temp["axis2"].quantile(0.25))
        z_IQR = (df_temp["axis3"].quantile(0.75) - df_temp["axis3"].quantile(0.25))

        # negtive count
        x_neg_count = (df_temp["axis1"] < 0).sum().sum()
        y_neg_count = (df_temp["axis2"] < 0).sum().sum()
        z_neg_count = (df_temp["axis3"] < 0).sum().sum()

        # positive count
        x_pos_count = (df_temp["axis1"] > 0).sum().sum()
        y_pos_count = (df_temp["axis2"] > 0).sum().sum()
        z_pos_count = (df_temp["axis3"] > 0).sum().sum()

        # values above mean
        x_above_mean = (df_temp["axis1"] > df_temp["axis1"].mean()).sum().sum()
        y_above_mean = (df_temp["axis2"] > df_temp["axis2"].mean()).sum().sum()
        z_above_mean = (df_temp["axis3"] > df_temp["axis3"].mean()).sum().sum()

        # number of peaks
        x_peak_count = len(find_peaks(df_temp["axis1"])[0])
        y_peak_count = len(find_peaks(df_temp["axis2"])[0])
        z_peak_count = len(find_peaks(df_temp["axis3"])[0])

        # skewness
        x_skewness = stats.skew(df_temp["axis1"])
        y_skewness = stats.skew(df_temp["axis2"])
        z_skewness = stats.skew(df_temp["axis3"])

        # kurtosis
        x_kurtosis = stats.kurtosis(df_temp["axis1"])
        y_kurtosis = stats.kurtosis(df_temp["axis2"])
        z_kurtosis = stats.kurtosis(df_temp["axis3"])

        # energy
        x_energy = ((df_temp["axis1"] **2) / 100).sum()
        y_energy = ((df_temp["axis2"] **2) / 100).sum()
        z_energy = ((df_temp["axis3"] **2) / 100).sum()

        # avg resultant
        avg_result_accl = ((df_temp["axis1"] ** 2 + df_temp["axis2"] ** 2 + df_temp["axis3"] ** 2) ** 0.5).mean()

        # signal magnitude area
        sma = (df_temp["axis1"].abs() / 100).sum() +(df_temp["axis2"].abs() / 100).sum() + (df_temp["axis3"].abs() / 100).sum()
        
        


        acc_summary = []
        acc_summary.extend([x_mean , y_mean, z_mean, x_std, y_std, z_std, x_aad, y_aad, z_aad, x_min, y_min, z_min, x_max, y_max, z_max, 
                            x_maxmin_diff, y_maxmin_diff, z_maxmin_diff, x_median, y_median, z_median, x_mad, y_mad, z_mad, x_IQR, y_IQR, z_IQR,
                            x_neg_count, y_neg_count, z_neg_count, x_pos_count, y_pos_count, z_pos_count,x_above_mean, y_above_mean, z_above_mean,
                            x_peak_count, y_peak_count, z_peak_count, x_skewness, y_skewness, z_skewness, x_kurtosis, y_kurtosis, z_kurtosis, x_energy,
                            y_energy, z_energy, avg_result_accl, sma, steps, inclineStanding, inclineSitting, inclineLying])
        


        X_train_temp = pd.DataFrame([acc_summary], columns = ["x_mean" , "y_mean", "z_mean", "x_std", "y_std", "z_std", "x_aad", "y_aad", "z_aad", "x_min", "y_min", "z_min", "x_max", "y_max", "z_max", 
                                                              "x_maxmin_diff", "y_maxmin_diff", "z_maxmin_diff", "x_median", "y_median", "z_median", "x_mad", "y_mad", "z_mad", "x_IQR", "y_IQR", "z_IQR",
                                                              "x_neg_count", "y_neg_count", "z_neg_count", "x_pos_count", "y_pos_count", "z_pos_count","x_above_mean", "y_above_mean", "z_above_mean",
                                                              "x_peak_count", "y_peak_count", "z_peak_count", "x_skewness", "y_skewness", "z_skewness", "x_kurtosis", "y_kurtosis", "z_kurtosis", "x_energy",
                                                              "y_energy", "z_energy", "avg_result_accl", "sma", "steps", "inclineStanding", "inclineSitting", "inclineLying"])

        
        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)
        
    return X_train

In [ ]:
def feat_statistical_measures_gl_acc_ml(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        cgm_summary = list(cgm.summary(df_temp))
        cgm_J_index = cgm.J_index(df_temp)
        cgm_interdaycv = cgm.interdaycv(df_temp)

        cgm_summary.append(cgm_J_index)
        cgm_summary.append(cgm_interdaycv)


        z_mean = df_temp["axis3"].mean()
        y_max = df_temp["axis2"].max()
        z_energy = ((df_temp["axis3"] ** 2) / 100).sum()

        acc_summary = []
        acc_summary.extend(
            [z_mean, y_max, z_energy])

        features = cgm_summary
        features.extend(acc_summary)

        X_train_temp = pd.DataFrame([features],
                                    columns=["mean", "median", "minimum", "maximum", "first_quartile", "third_quartile",
                                             "J_index", "cgm_interdaycv", "z_mean", "y_max", "z_energy"])

        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)

    return X_train

In [ ]:
def feat_statistical_measures_acc_ml(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        # mean
        z_mean = df_temp["axis3"].mean()
        y_max = df_temp["axis2"].max()
        z_energy = ((df_temp["axis3"] ** 2) / 100).sum()


        acc_summary = []
        acc_summary.extend(
            [z_mean, y_max, z_energy])

        X_train_temp = pd.DataFrame([acc_summary],
                                    columns=["z_mean", "y_max", "z_energy"])

        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)

    return X_train


In [ ]:
def feat_statistical_measures_gl_ml(x_list):
    X_train = pd.DataFrame()

    for df_temp in x_list:
        cgm_summary = list(cgm.summary(df_temp))
        cgm_LBGI = cgm.LBGI(df_temp)
        cgm_HBGI = cgm.HBGI(df_temp)
        cgm_ADRR = cgm.ADRR(df_temp)
        cgm_GMI = cgm.GMI(df_temp)
        cgm_J_index = cgm.J_index(df_temp)
        cgm_eA1c = cgm.eA1c(df_temp)
        cgm_interdaysd = cgm.interdaysd(df_temp)
        cgm_interdaycv = cgm.interdaycv(df_temp)
        cgm_TOR = cgm.TOR(df_temp, sd=1, sr=15)
        cgm_TIR = cgm.TIR(df_temp, sd=1, sr=15)
        cgm_POR = cgm.POR(df_temp, sd=1, sr=15)


        cgm_summary.append(cgm_LBGI)
        cgm_summary.append(cgm_HBGI)
        cgm_summary.append(cgm_ADRR)
        cgm_summary.append(cgm_GMI)
        cgm_summary.append(cgm_J_index)
        cgm_summary.append(cgm_eA1c)
        cgm_summary.append(cgm_interdaysd)
        cgm_summary.append(cgm_interdaycv)
        cgm_summary.append(cgm_TOR)
        cgm_summary.append(cgm_TIR)
        cgm_summary.append(cgm_POR)

        X_train_temp = pd.DataFrame([cgm_summary],
                                    columns=["mean", "median", "minimum", "maximum", "first_quartile", "third_quartile",
                                             "LBGI", "HBGI", "ADRR", "GMI", "J_index", "eA1c", "interdaysd",
                                             "cgm_interdaycv",
                                             "cgm_TOR", "cgm_TIR",
                                             "cgm_POR"])

        X_train = pd.concat([X_train, X_train_temp], ignore_index=True)

    return X_train

In [ ]:

X_test = feat_statistical_measures_gl_acc(x_list_test)

X_train = feat_statistical_measures_gl_acc(x_list_train)

In [ ]:

X_test = feat_statistical_measures_gl_ml( x_list_test)

X_train = feat_statistical_measures_gl_ml(x_list_train)

In [ ]:

X_test = feat_statistical_measures_gl_acc_ml( x_list_test)

X_train = feat_statistical_measures_gl_acc_ml(x_list_train)

In [ ]:

X_test = feat_statistical_measures_acc_ml( x_list_test)

X_train = feat_statistical_measures_acc_ml(x_list_train)

In [ ]:
X_test

In [ ]:
X_test.to_csv("ML_data/eval_X_test_acc.csv", sep='\t')
X_train.to_csv("ML_data/eval_X_train_acc.csv", sep='\t')


In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [ ]:
def to_float(input_string):
    if input_string == "fasting":
        return 0
    else:
        return 1

In [ ]:
func_float = np.vectorize(to_float)
y_train_float = func_float(y_train)
y_test_float = func_float(y_test)

In [ ]:
np.save('ML_data/y_train_eval_acc.npy', y_train_float)
np.save('ML_data/y_test_eval_acc.npy', y_test_float)

In [ ]:
features_to_keep = ["cgm_interdaycv", "J_index", "maximum", "z_mean", "y_max", "z_energy"]

X_test = X_test[features_to_keep] 
X_train = X_train[features_to_keep] 

In [ ]:
features_to_keep = ["cgm_interdaycv", "J_index", "maximum"]

X_test = X_test[features_to_keep] 
X_train = X_train[features_to_keep] 

In [ ]:
features_to_keep = ["z_mean", "y_max", "z_energy"]

X_test = X_test[features_to_keep] 
X_train = X_train[features_to_keep] 

# Feature Selection

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=1), n_features_to_select=4)
fit = rfe.fit(X_train, y_train_float)


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier

rfe = RFE(DecisionTreeClassifier(), n_features_to_select=6)
fit = rfe.fit(X_train, y_train_float)
# report selected features
print('Selected Features:')

In [ ]:
fit.ranking_

In [ ]:
names = X_train.columns.values[0:-1]
names

In [ ]:
X_train['y_max'].corr(X_train['z_mean'])

## Balancing

https://imbalanced-learn.org/stable/over_sampling.html

In [ ]:
balanced_df = final_df.drop(["Time", "time", "Day"], axis=1)

balanced_df.phase = pd.Categorical(balanced_df.phase)
balanced_df['phase'] = balanced_df.phase.cat.codes

balanced_df.label = pd.Categorical(balanced_df.label)
balanced_df['label'] = balanced_df.label.cat.codes


In [ ]:
X_test = pd.read_csv('eval_X_test.csv', sep='\t')
X_train = pd.read_csv('eval_X_train.csv', sep='\t')

X_test = X_test.drop(["Unnamed: 0"], axis=1)
X_train = X_train.drop(["Unnamed: 0"], axis=1)

In [ ]:
y_train_float = np.load('ML_data/y_train_eval.npy')
y_test_float = np.load('ML_data/y_test_eval.npy')

In [ ]:
X_test

In [ ]:
print(sorted(collections.Counter(y_test_float).items()))
print(sorted(collections.Counter(y_train_float).items()))

In [ ]:
from imblearn.combine import SMOTETomek
import collections

In [ ]:

smote_tomek = SMOTETomek(random_state=0)
X_test_resampled, y_test_resampled = smote_tomek.fit_resample(X_test, y_test_float)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train, y_train_float)


In [ ]:
print(sorted(collections.Counter(y_test_resampled).items()))
print(sorted(collections.Counter(y_train_resampled).items()))


In [ ]:
X_train_resampled.values

In [ ]:
X_train_resampled

## Logistic Regression

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train_resampled)

X_train_data_lr = pd.DataFrame(scaler.transform(X_train_resampled), index=X_train_resampled.index, columns=X_train_resampled.columns)
X_test_data_lr = pd.DataFrame(scaler.transform(X_test_resampled), index=X_test_resampled.index, columns=X_test_resampled.columns)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train_resampled)
X_train_data_lr = scaler.transform(X_train_resampled)
X_test_data_lr = scaler.transform(X_test_resampled)


In [ ]:
# logistic regression model
lr = LogisticRegression(penalty='l2', solver='liblinear', max_iter=157, C=0.97)
lr.fit(X_train_data_lr, y_train_resampled)
y_pred = lr.predict(X_test_data_lr)
y_pred = smoothing(y_pred)
print("Accuracy:", accuracy_score(y_test_resampled, y_pred))
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, y_pred))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()

#### Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
estimator = LogisticRegression(
    random_state = 1,
    penalty = 'l2'
)
#estimator.fit(X_train_data_lr, y_train_resampled)

In [ ]:
max_iter = range(100, 200)
solver = ['lbfgs', 'newton-cg', 'liblinear']
warm_start = [True, False]
C = np.arange(0, 1, 0.01)
random_grid ={
    'max_iter' : max_iter,
    'warm_start' : warm_start,
    'solver' : solver,
    'C' : C,
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
random_estimator = RandomizedSearchCV(estimator = estimator,
                                   param_distributions = random_grid,
                                   n_iter = 100,
                                   scoring = 'accuracy',
                                   n_jobs = -1,
                                   verbose = 1, 
                                   random_state = 1,
                                  )

random_estimator.fit(X_train_data_lr, y_train_resampled)

In [ ]:
random_estimator.best_params_

In [ ]:
random_estimator.best_params_

best_estimator = random_estimator.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(y_test_resampled,pred)

In [ ]:
y_pred_smoothed = smoothing(pred)
accuracy_score(y_test_resampled,y_pred_smoothed)

In [ ]:
print(metrics.classification_report(y_test_resampled, pred))

#### Evaluation

In [ ]:
def GridSearch_table_plot(grid_clf, param_name,
                          num_results=15,
                          negative=True,
                          graph=True,
                          display_all_params=True):

    '''Display grid search results

    Arguments
    ---------

    grid_clf           the estimator resulting from a grid search
                       for example: grid_clf = GridSearchCV( ...

    param_name         a string with the name of the parameter being tested

    num_results        an integer indicating the number of results to display
                       Default: 15

    negative           boolean: should the sign of the score be reversed?
                       scoring = 'neg_log_loss', for instance
                       Default: True

    graph              boolean: should a graph be produced?
                       non-numeric parameters (True/False, None) don't graph well
                       Default: True

    display_all_params boolean: should we print out all of the parameters, not just the ones searched for?
                       Default: True

    Usage
    -----

    GridSearch_table_plot(grid_clf, "min_samples_leaf")

                          '''
    from matplotlib      import pyplot as plt
    from IPython.display import display
    import pandas as pd

    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')

    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    # =====================================
    display(pd.DataFrame(cv_results) \
            .sort_values(by='rank_test_score').head(num_results))

    # plot the results
    # ================
    scores_df = scores_df.sort_values(by='param_' + param_name)

    if negative:
        means = -scores_df['mean_test_score']
    else:
        means = scores_df['mean_test_score']
    stds = scores_df['std_test_score']
    params = scores_df['param_' + param_name]

    # plot
    if graph:
        plt.figure(figsize=(8, 8))
        plt.errorbar(params, means, yerr=stds)

        plt.axhline(y=best_mean + best_stdev, color='red')
        plt.axhline(y=best_mean - best_stdev, color='red')
        plt.plot(best_param, best_mean, 'or')

        plt.title(param_name + " vs Score\nBest Score {:0.5f}".format(clf_score))
        plt.xlabel(param_name)
        plt.ylabel('Score')
        plt.show()

In [ ]:
GridSearch_table_plot(random_estimator, "C", negative=False)

In [ ]:
random_estimator.cv_results_['params']

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
import pickle
with open('ML_data/models_ML/gl/ML_LR_adapt_scaler.pkl','wb') as f:
    pickle.dump(lr,f)

In [ ]:
import pickle
with open('LR_random_estimator.pkl','wb') as f:
    pickle.dump(random_estimator,f)

In [ ]:
pickle.dump(scaler, open("ML_data/models_ML/gl/ML_scaler_adapt.pkl", 'wb'))

## SVM

In [ ]:
from sklearn import svm

svm_clf = svm.SVC(C=1000.0, kernel='rbf')
svm_clf.fit(X_train_data_lr, y_train_resampled)


In [ ]:
y_pred = svm_clf.predict(X_test_data_lr)


In [ ]:
print("Accuracy:", accuracy_score(y_test_resampled, y_pred))
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, y_pred))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
import shap
# Fits the explainer
explainer = shap.Explainer(svm_clf.predict, X_test_data_lr)
# Calculates the SHAP values - It takes some time
shap_values = explainer(X_test_data_lr)


In [ ]:
import pickle
with open('ML_data/models_ML/gl/ML_SVM_adapt.pkl','wb') as f:
    pickle.dump(svm_clf,f)

### Random CV

In [ ]:
parameters = {'kernel': ['rbf', 'sigmoid', 'poly'],
             'C': [0.1, 1.0, 100.0, 1000.0],
             'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0],}#



In [ ]:
random_estimator = RandomizedSearchCV(estimator = svm_clf,
                                   param_distributions = parameters,
                                   cv = 5,
                                   n_iter = 100,
                                   scoring = 'accuracy',
                                   n_jobs = -1,
                                   verbose = 1, 
                                   random_state = 1,
                                  )

random_estimator.fit(X_train_data_lr, y_train_resampled)

In [ ]:
random_estimator.best_params_

best_estimator = random_estimator.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(pred, y_test_resampled)

In [ ]:
y_pred_smoothed = smoothing(pred)
accuracy_score(y_pred_smoothed, y_test_resampled)

In [ ]:
GridSearch_table_plot(random_estimator, "gamma", negative=False)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test_resampled, pred))

## SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier
sdg_clf = SGDClassifier(loss="log", penalty="l2", alpha = 0.001, learning_rate='optimal')
sdg_clf.fit(X_train_data_lr, y_train_resampled)
y_pred = sdg_clf.predict(X_test_data_lr)
print("Accuracy:", accuracy_score(y_test_resampled, smoothing(y_pred)))
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled,  smoothing(y_pred)))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
import pickle
with open('ML_data/models_ML/gl/ML_SDG_adapt.pkl','wb') as f:
    pickle.dump(sdg_clf,f)

In [ ]:
# set search parameters
loss = ['hinge', 'log_loss', 'log', 'modified_huber', 'perceptron', 'squared_error']
penalty = ['l2','l1','elasticnet']
learning_rate = ['constant','optimal','invscaling']
alpha = [0.0001, 0.001, 0.01, 0.1]
random_grid = {
    'loss': loss,
    'penalty': penalty,
    'alpha': alpha,
    'learning_rate': learning_rate
}

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier() 
sgd_random = RandomizedSearchCV(estimator = sgd, random_state = 42,n_jobs = -1,param_distributions = random_grid,n_iter = 100, cv=3,verbose = 2)
sgd_random.fit(X_train_data_lr, y_train_resampled)
sgd_random.best_params_

In [ ]:
sgd_random.best_params_

best_estimator = sgd_random.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(pred, y_test_resampled)

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, pred))

In [ ]:
GridSearch_table_plot(sgd_random, "loss", negative=False)

## KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=13,weights="distance", metric="manhattan" )
#model = KNeighborsClassifier( )

# Train the model using the training sets
model.fit(X_train_data_lr,y_train_resampled)
y_pred = model.predict(X_test_data_lr)
print("Accuracy:", accuracy_score(y_test_resampled, smoothing(y_pred)))
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, smoothing(y_pred)))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled,smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
import pickle
with open('ML_data/models_ML/gl/ML_KNN_adapt.pkl','wb') as f:
    pickle.dump(model,f)

In [ ]:
# set search parameters
n_neighbors = [int(x) for x in np.linspace(start = 1, stop = 100, num = 50)]   
weights = ['uniform','distance']
metric = ['euclidean','manhattan','chebyshev','minkowski'] 
random_grid = {
    'n_neighbors': n_neighbors,
    'weights': weights,
    'metric': metric,
}

In [ ]:
knn = KNeighborsClassifier() 
knn_random = RandomizedSearchCV(estimator = knn, random_state = 42,n_jobs = -1,param_distributions = random_grid,n_iter = 100, cv=3,verbose = 2)
knn_random.fit(X_train_data_lr, y_train_resampled)
knn_random.best_params_

In [ ]:
knn_random.best_params_

best_estimator = knn_random.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(pred, smoothing(y_test_resampled))

In [ ]:
GridSearch_table_plot(knn_random, "metric", negative=False)

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, pred))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


regressor = RandomForestClassifier(n_estimators=40, random_state=0, max_depth=20, bootstrap=False)
regressor.fit(X_train_data_lr, y_train_resampled)
y_pred = regressor.predict(X_test_data_lr)
print("Accuracy:",metrics.accuracy_score(y_test_resampled, smoothing(y_pred)))

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, smoothing(y_pred)))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 40, num = 40)] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rf = RandomForestClassifier(random_state = 35)


In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
rf_random.fit(X_train_data_lr,y_train_resampled)


In [ ]:
rf_random.best_params_

best_estimator = rf_random.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(pred, y_test_resampled)

In [ ]:
GridSearch_table_plot(rf_random, "max_depth", negative=False)

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled,pred))

In [ ]:
import pickle
with open('ML_data/models_ML/gl/ML_RF_adapt.pkl','wb') as f:
    pickle.dump(regressor,f)

## Decision Tree

In [ ]:
from sklearn import tree

dt_clf = tree.DecisionTreeClassifier(criterion="gini", max_depth=20, min_samples_leaf= 100, max_features="sqrt")
dt_clf = dt_clf.fit(X_train_data_lr, y_train_resampled)
y_pred = dt_clf.predict(X_test_data_lr)
print("Accuracy:", accuracy_score(y_test_resampled, smoothing(y_pred)))
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, smoothing(y_pred)))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [int(x) for x in np.linspace(10, 120, num = 12)],
    'min_samples_leaf': [5, 10, 20, 50, 100, 150],
    'max_features' : ['auto', 'sqrt'],
    'criterion': ["gini", "entropy"]
}

In [ ]:
from sklearn import tree

dt = DecisionTreeClassifier(random_state=42)

In [ ]:
dt_random = RandomizedSearchCV(estimator = dt, param_distributions = params,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
dt_random.fit(X_train_data_lr,y_train_resampled)


In [ ]:
dt_random.best_params_

best_estimator = dt_random.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(pred, y_test_resampled)

In [ ]:
GridSearch_table_plot(dt_random, "min_samples_leaf", negative=False)

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled,pred))

In [ ]:
import pickle
with open('ML_data/models_ML/gl/ML_DT_adapt.pkl','wb') as f:
    pickle.dump(dt_clf,f)

## Multi-layer perceptron (MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(solver='adam', alpha=0.0001, hidden_layer_sizes=(50, 50, 50), random_state=1)
mlp_clf.fit(X_train_data_lr, y_train_resampled)
y_pred = mlp_clf.predict(X_test_data_lr)


In [ ]:
print("Accuracy:", accuracy_score(y_test_resampled, smoothing(y_pred)))
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, smoothing(y_pred)))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=100)
mlp.fit(X_train_data_lr, y_train_resampled)


parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
random_estimator = RandomizedSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
random_estimator.fit(X_train_data_lr, y_train_resampled)

In [ ]:
random_estimator.best_params_

best_estimator = random_estimator.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(pred, y_test_resampled)

In [ ]:
GridSearch_table_plot(random_estimator, "alpha", negative=False)

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled,smoothing(y_pred))

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, pred))

In [ ]:
import pickle
with open('ML_data/models_ML/gl/ML_MLP_adapt.pkl','wb') as f:
    pickle.dump(mlp_clf,f)

In [ ]:
X_test_data_lr

In [ ]:
print(X_test_data_lr[:5])

In [ ]:
shap.plots.waterfall(shap_values[1456])

In [ ]:
# Fits the explainer
explainer = shap.Explainer(svm_clf.predict, X_test_data_lr)
# Calculates the SHAP values - It takes some time
shap_values = explainer(X_test_data_lr[:400])

In [ ]:
import shap

shap.summary_plot(shap_values, feature_names=["glucose interdaycv", "glucose J_index", "glucose maximum", "acceleration z_mean", "acceleration y_max", "acceleration z_energy"])
